In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
df = pd.read_csv("credit card.csv")

In [9]:
df.head()

,Unnamed: 0,Customer_ID,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,A_10,A_11,A_12,A_13,A_14,class
0,0,15776156,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,1,15739548,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,2,15662854,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,3,15687688,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,4,15715750,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   690 non-null    int64  
 1   Customer_ID  690 non-null    int64  
 2   A_1          690 non-null    int64  
 3   A_2          690 non-null    float64
 4   A_3          690 non-null    float64
 5   A_4          690 non-null    int64  
 6   A_5          690 non-null    int64  
 7   A_6          690 non-null    int64  
 8   A_7          690 non-null    float64
 9   A_8          690 non-null    int64  
 10  A_9          690 non-null    int64  
 11  A_10         690 non-null    int64  
 12  A_11         690 non-null    int64  
 13  A_12         690 non-null    int64  
 14  A_13         690 non-null    int64  
 15  A_14         690 non-null    int64  
 16  class        690 non-null    int64  
dtypes: float64(3), int64(14)
memory usage: 91.8 KB


In [11]:
df.shape

(690, 17)

THIS DATASET IS HIGHLY UNBALANCED

   0 -> NORMAL TRANSACTION,
   1 -> FRAUDULENT TRANSACTION

**SEPARATING DATA FOR ANALYSIS**

In [ ]:
legit = df[df.Class == 0]
fraud = df[df.Class == 1]

**STATISTICAL MEASURE OF DATA**

In [ ]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [ ]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

COMPARE THE VALUES FOR BOTH TRANSACTIONS

In [ ]:
df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


**UNDER SAMPLING**

BUILD A SAMPLE DATASET CONTAINING SIMILAR DISTRIBUTION OF NORMAL TRANSACTIONS AND FRAUDULENT TRANSACTIONS

NUMBER OF FRAUDULENT TRANSACTIONS -> 492

In [ ]:
legit_sample = legit.sample(n=492)

**CONCATENATING TWO DATAFRAMES**

In [ ]:
new_df = pd.concat([legit_sample , fraud], axis = 0)

In [ ]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5253,5087.0,0.989228,-0.593941,1.071029,0.194283,-0.824501,0.596833,-0.845974,0.237445,2.105335,...,-0.156802,-0.201216,-0.030602,-0.290607,0.002989,0.975035,-0.064079,0.008407,95.72,0
165292,117326.0,-0.200561,0.027684,-0.241402,-1.992654,2.460482,3.890078,-0.332155,1.127142,0.441805,...,0.309899,0.896232,0.014889,0.749680,-0.978877,0.337641,0.218154,0.228672,11.50,0
52940,45725.0,-0.671622,0.881131,1.608816,-0.383154,0.499870,-0.636383,0.699571,0.034465,-0.571393,...,-0.214499,-0.728547,-0.097145,0.023041,0.230822,0.047186,-0.061347,-0.016292,4.47,0
262352,160433.0,2.111519,-0.036044,-2.260689,0.089109,0.868772,-0.528819,0.472292,-0.215171,0.047921,...,0.110263,0.425802,-0.045916,0.280934,0.384318,0.696027,-0.124466,-0.095338,0.76,0
92946,64180.0,0.790185,-0.691863,0.999747,1.567220,-1.259973,-0.227572,-0.392125,0.093115,0.899874,...,0.225601,0.318288,-0.244231,0.368881,0.388970,-0.258061,0.022267,0.066770,200.00,0


In [ ]:
new_df["Class"].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [ ]:
new_df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94900.554878,0.020853,0.057981,0.023200,-0.010597,0.047686,0.048002,0.093713,0.060434,0.073293,...,-0.040427,0.005700,0.015350,0.027177,-0.034146,-0.036371,-0.015758,-0.014608,0.001680,82.566423
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


**SPLITTING THE DATA INTO FEATURES AND TARGETS**

In [ ]:
X = new_df.drop(columns = "Class", axis=1)
Y = new_df["Class"]

In [ ]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
5253      5087.0  0.989228 -0.593941  1.071029  0.194283 -0.824501  0.596833   
165292  117326.0 -0.200561  0.027684 -0.241402 -1.992654  2.460482  3.890078   
52940    45725.0 -0.671622  0.881131  1.608816 -0.383154  0.499870 -0.636383   
262352  160433.0  2.111519 -0.036044 -2.260689  0.089109  0.868772 -0.528819   
92946    64180.0  0.790185 -0.691863  0.999747  1.567220 -1.259973 -0.227572   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [ ]:
print(Y)

5253      0
165292    0
52940     0
262352    0
92946     0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


**SPLITTING DATA INTO TRAINING DATA AND TESTING DATA**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


**MODEL TRAINING**

LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression()

TRAINING THE LOGISTIC REGRESSION MODEL WITH TRAINING DATA

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

**MODEL EVALUAION**

ACCURACY SCORE

ACCURACY ON TRAINING DATA

In [ ]:
X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_train)

In [ ]:
print("ACCURACY ON TRAINING DATA:", training_data_accuracy)

ACCURACY ON TRAINING DATA: 0.9453621346886912


ACCURACY ON TESTING DATA

In [ ]:
X_test_predict = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_predict, Y_test)

In [ ]:
print("ACCURACY ON TESTING DATA:", testing_data_accuracy)

ACCURACY ON TESTING DATA: 0.9289340101522843
